In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import numpy as np
from io import StringIO
from tqdm import tqdm_notebook

## Сбор игр

In [2]:
old_df = pd.read_csv('raw_games.csv')
old_df.head()

,name,number,game_id,place,date
0,[music party],#9,65996,Old School Pub,"1 ноября, Среда"
1,[мультики],#8,65997,Papa' S Bar & Grill,"1 ноября, Среда"
2,[новички],#899,65984,Резиденция,"1 ноября, Среда"
3,"Квиз, плиз!",#899,65994,Chesterfield Bar,"1 ноября, Среда"
4,[HALLOWEEN],#3,65986,Резиденция,"31 октября, Вторник"


In [7]:
games = []

for i in range(80, 100):
    bs = BeautifulSoup(requests.get(f'https://quizplease.ru/schedule-past?page={i}&per-page=12').text)
    sleep(np.random.uniform(3, 5))
    for block in bs.find_all('div', {'class': 'schedule-column'}):
        name, number = [b.text for b in block.find_all('div', {'class': 'h2-game-card'})]
        game_id = block.find_all('a', {'class': 'schedule-block-head'})[0]['href'].split('id=')[-1]
        place = block.find_all('div', {'class': 'schedule-block-info-bar'})[0].text.replace('Где это?', '').strip()
        date = block.find_all('div', {'class': 'h3'})[0].text

        games.append({
            "name": name,
            "number": number,
            "game_id": game_id,
            "place": place, 
            "date": date
        })

In [8]:
new_df = pd.DataFrame(games)
new_df['game_id'] = new_df['game_id'].astype(int)
print(new_df.shape)
new_df = new_df[~new_df['game_id'].isin(old_df['game_id'])]
print(new_df.shape)

(240, 5)
(240, 5)


In [9]:
df = pd.concat([old_df, new_df], axis=0, ignore_index=True)

In [10]:
df.to_csv('raw_games.csv', index=False)

## Парсим игры

In [11]:
df = pd.read_csv('./raw_games.csv')
game_df = pd.read_csv('./games.csv')

/home/ilya/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (5,19,29,30,32,33,34,43,46) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
game_dfs = []

In [13]:
df[~df['game_id'].isin(game_df['game_id'].astype(int))].shape

(245, 5)

In [14]:
for i, row in tqdm_notebook(df[~df['game_id'].isin(game_df['game_id'].astype(int))].iterrows()):
    game_id = row["game_id"]
    try:
        url = f'https://quizplease.ru/game-page?id={game_id}'
        r = requests.get(url)
        soup = BeautifulSoup(r.text)

        tb = soup.find_all('table', {'class': 'game-table'})[0]

        new_df = pd.read_html(StringIO(str(tb)))[0]
        new_df['game_id'] = game_id
        game_dfs.append(new_df)
        df.loc[i, 'parsed'] = True
        sleep(np.random.uniform(2, 4))
    except IndexError:
        print(url)

/home/ilya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


https://quizplease.ru/game-page?id=65205
https://quizplease.ru/game-page?id=61464
https://quizplease.ru/game-page?id=59652
https://quizplease.ru/game-page?id=58876
https://quizplease.ru/game-page?id=55931



In [15]:
new_game_dfs = pd.concat(game_dfs, axis=0, ignore_index=True)
new_game_dfs = new_game_dfs[~new_game_dfs['game_id'].isin(game_df['game_id'])]

In [16]:
game_df = pd.concat([game_df, new_game_dfs], axis=0, ignore_index=True)

In [17]:
game_df['game_id'] = game_df['game_id'].astype(int)

In [18]:
game_df.to_csv('./games.csv', index=False)

In [19]:
our_df = game_df[game_df['Название команды'] == 'Москва-Петушок Большая Кокоша']
print(our_df.shape)

(21, 53)


In [20]:
our_df = our_df.drop(our_df.isnull().mean().loc[lambda x: x == 1].index, axis=1)

In [21]:
our_df.merge(df, on='game_id').sort_values('game_id').set_index('date')

,Место,Название команды,Итого,1 раунд,2 раунд,3 раунд,4 раунд,5 раунд,6 раунд,7 раунд,game_id,итого,name,number,place,parsed
date,,,,,,,,,,,,,,,,
"5 марта, Воскресенье",45.0,Москва-Петушок Большая Кокоша,NaN,4.0,5.0,1.0,8.0,2,0.0,2.0,52762,22.0,"Квиз, плиз!",#796,Chesterfield Bar,True
"26 мая, Пятница",12.0,Москва-Петушок Большая Кокоша,NaN,3.0,4.0,4.0,3.0,6.0,4.0,4.0,57366,28.0,"Квиз, плиз!",#831,Резиденция,NaN
"15 июня, Четверг",13.0,Москва-Петушок Большая Кокоша,NaN,5,3.0,5.0,9.0,4.0,0.0,12.0,58490,38.0,"Квиз, плиз!",#840,Chesterfield Bar,NaN
"6 июля, Четверг",30.0,Москва-Петушок Большая Кокоша,NaN,6.0,3.0,4.0,5.0,3.0,2.0,9.0,59660,32.0,"Квиз, плиз!",#849,Chesterfield Bar,NaN
"18 июля, Вторник",2.0,Москва-Петушок Большая Кокоша,NaN,3.0,5.0,6.0,10.0,2.0,4.0,13.0,60284,43.0,"Квиз, плиз!",#854,Резиденция,NaN
"27 июля, Четверг",11.0,Москва-Петушок Большая Кокоша,NaN,4.0,5.0,5.0,4.0,4.5,2.0,11.0,60685,35.5,[кино и музыка],#137,MT Music Bar,NaN
"3 августа, Четверг",31.0,Москва-Петушок Большая Кокоша,NaN,6.0,4.0,3.0,1.0,3.0,4.0,11.0,61072,32.0,"Квиз, плиз!",#861,Chesterfield Bar,NaN
"9 августа, Среда",35.0,Москва-Петушок Большая Кокоша,30.0,5.0,5.0,4.0,5.0,4.0,0.0,7.0,61481,NaN,"Квиз, плиз!",#863,Chesterfield Bar,NaN
"15 августа, Вторник",24.0,Москва-Петушок Большая Кокоша,NaN,5.0,4.0,4.0,8.0,3.0,2.0,10.0,61675,36.0,"Квиз, плиз!",#866,Chesterfield Bar,NaN
